In [1]:
directory = r"c:\Users\root\Desktop\drive-download-20250516T195333Z-1-001"

In [2]:
from PIL import Image
from PIL.ExifTags import TAGS
import os
from datetime import datetime
import pillow_heif
pillow_heif.register_heif_opener()


def get_image_date(image_path):
    """Extracts the date the image was taken from EXIF metadata."""
    try:
        with Image.open(image_path) as img:
            exif_data = img.getexif()
            if exif_data:
                tag_dict = {TAGS.get(tag, tag): value for tag, value in exif_data.items()}
                # Try to return the most reliable tag
                return (
                    #COME BACK HERE IF SOMETHING DOES NOT WORK
                    # tag_dict.get("DateTimeOriginal") or
                    # tag_dict.get("DateTimeDigitized") or
                    tag_dict.get("DateTime") or
                    "No date"
                )
    except Exception as e:
        print(f"Error reading {image_path}: {e}")
    return "No date"

def format_date(date_string):
    """Formats the date as IMG_[Y][M][D]_[H][M][S]."""
    try:
        dt = datetime.strptime(date_string, "%Y:%m:%d %H:%M:%S")
        return dt.strftime("IMG_%Y%m%d_%H%M%S")
    except ValueError:
        return "Invalid date"

In [3]:
def list_images_with_dates(directory):
    """Lists all images in the directory with their capture dates in the format IMG_[Y][M][D]_[H][M][S]."""
    valid_extensions = ('.jpg', '.jpeg', '.png', '.tiff', '.heic')
    
    for filename in os.listdir(directory):
        if filename.lower().endswith(valid_extensions):
            image_path = os.path.join(directory, filename)
            date_created = get_image_date(image_path)
            formatted_date = format_date(date_created) if date_created != "No date" else date_created
            print(f"{filename}: {formatted_date}")

if os.path.exists(directory):
    list_images_with_dates(directory)
else:
    print("Invalid directory!")


0EB903B2-0B56-4634-BCFA-6C4D2F5AF54A.jpg: IMG_20250425_163604
IMG_8908.HEIC: IMG_20250423_131251
IMG_8909.HEIC: IMG_20250423_131319
IMG_8910.HEIC: IMG_20250423_131356
IMG_8946.HEIC: IMG_20250425_154239
IMG_8947.HEIC: IMG_20250425_154241
IMG_8950.HEIC: IMG_20250425_173931
IMG_8959.HEIC: IMG_20250426_093728
IMG_8960.HEIC: IMG_20250426_093730
IMG_8961.HEIC: IMG_20250426_093731
IMG_8962.HEIC: IMG_20250426_093736
IMG_8963.HEIC: IMG_20250426_093738
IMG_8964.HEIC: IMG_20250426_134225
IMG_8965.HEIC: IMG_20250426_134236
IMG_8966.HEIC: IMG_20250426_134249
IMG_8967.HEIC: IMG_20250426_134537
IMG_8969.HEIC: IMG_20250426_203556
IMG_8970.HEIC: IMG_20250426_211012
IMG_8971.HEIC: IMG_20250426_211026
IMG_8972.HEIC: IMG_20250426_211714
IMG_8973.HEIC: IMG_20250426_211726
IMG_8974.HEIC: IMG_20250426_232005
IMG_8975.HEIC: IMG_20250426_232007
IMG_8976.HEIC: IMG_20250426_232007
IMG_8978.HEIC: IMG_20250427_100805
IMG_8979.HEIC: IMG_20250427_100809
IMG_8980.HEIC: IMG_20250427_120339
IMG_8981.HEIC: IMG_20250427_

In [5]:
def rename_images(directory):
    """Renames images in a directory based on their capture date."""
    valid_extensions = ('.jpg', '.jpeg', '.png', '.tiff', '.heic')
    
    for filename in os.listdir(directory):
        if filename.lower().endswith(valid_extensions):
            image_path = os.path.join(directory, filename)
            date_created = get_image_date(image_path)
            
            if date_created:
                formatted_date = format_date(date_created)
                if formatted_date:
                    new_filename = formatted_date + os.path.splitext(filename)[1]
                    for i in range(1, 100):
                        try:
                            new_path = os.path.join(directory, new_filename)
                            # print(f"Image: {image_path} -> New Path: {new_path}")
                            os.rename(image_path, new_path)
                            print(f"Renamed: {filename} -> {new_filename}")
                            break
                        except FileExistsError:
                            print(f"File {new_filename} already exists, trying with a number suffix.")
                            new_filename = new_filename.split('.')[0] + f"_{i}." + new_filename.split('.')[-1]
                            continue

                else:
                    print(f"Skipping {filename}: Invalid date format")
            else:
                print(f"Skipping {filename}: No date found")

if os.path.exists(directory):
    rename_images(directory)
else:
    print("Invalid directory!")


Renamed: IMG_20250423_131251.HEIC -> IMG_20250423_131251.HEIC
Renamed: IMG_20250423_131319.HEIC -> IMG_20250423_131319.HEIC
Renamed: IMG_20250423_131356.HEIC -> IMG_20250423_131356.HEIC
Renamed: IMG_20250425_154239.HEIC -> IMG_20250425_154239.HEIC
Renamed: IMG_20250425_154241.HEIC -> IMG_20250425_154241.HEIC
Renamed: IMG_20250425_163604.jpg -> IMG_20250425_163604.jpg
Renamed: IMG_20250425_173931.HEIC -> IMG_20250425_173931.HEIC
Renamed: IMG_20250426_093728.HEIC -> IMG_20250426_093728.HEIC
Renamed: IMG_20250426_093730.HEIC -> IMG_20250426_093730.HEIC
Renamed: IMG_20250426_093731.HEIC -> IMG_20250426_093731.HEIC
Renamed: IMG_20250426_093736.HEIC -> IMG_20250426_093736.HEIC
Renamed: IMG_20250426_093738.HEIC -> IMG_20250426_093738.HEIC
Renamed: IMG_20250426_134225.HEIC -> IMG_20250426_134225.HEIC
Renamed: IMG_20250426_134236.HEIC -> IMG_20250426_134236.HEIC
Renamed: IMG_20250426_134249.HEIC -> IMG_20250426_134249.HEIC
Renamed: IMG_20250426_134537.HEIC -> IMG_20250426_134537.HEIC
Renamed: I